In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import tensorflow as tf
from tensorflow import keras

import os
import cv2

import warnings
warnings.filterwarnings("ignore")

import sys
print("Python Version: ", sys.version)
print("TensorFlow Version: ", tf.__version__)

BATCH_SIZE = 32
IMG_WIDTH  = 224
IMG_HEIGHT = 224
IMAGE_SHOW = 100

%matplotlib inline
os.environ['KMP_DUPLICATE_LIB_OK']='True'

ModuleNotFoundError: No module named 'cv2'

In [ ]:
# load model
model = keras.models.load_model("../TrainingModels/furniture_cat_classification_model.h5")

In [ ]:
# using GPU for faster processing

phys_devices = tf.config.list_physical_devices('GPU')
if len(phys_devices) > 0:
    # use the first visible GPU
    tf.config.experimental.set_visible_devices(phys_devices[0], 'GPU')
    print(f"Using GPU device {phys_devices[0].name}\n")
else:
    print("No GPU devices found!\n")

if tf.config.list_physical_devices('GPU'):
    print("GPU is available")
else:
    print("GPU is not available")

In [ ]:
furniture_df = pd.read_csv("../furniture_data.csv")
# shuffle the rows randomly and reset the index
shuffled_furniture_df = furniture_df.sample(frac = 1).reset_index(drop = True)

shuffled_furniture_df.head()

In [ ]:
# prepare the test dataset
def prepare(filepath):
    image_arr = cv2.cvtColor(cv2.imread(filepath), cv2.COLOR_BGR2RGB)
    # resize image
    new_arr = cv2.resize(image_arr, (IMG_WIDTH, IMG_HEIGHT))
    return np.expand_dims(new_arr, axis = 0).repeat(BATCH_SIZE, axis = 0)

# empty list for test images
image_test_list = []

# add image for prediction testing
for i in range(IMAGE_SHOW):
     # get the file path for the i-th image
    first_data = shuffled_furniture_df.iloc[i]['Image_Path']
    image_arr = prepare(first_data)
    image_test_list.append(image_arr)

# check image after process
plt.imshow(img_test_list[19][0])

In [ ]:
cat_folders = furniture_df["Furniture_Category"].unique()
cat_folders

In [ ]:
# single prediction
index = 19
first_data = shuffled_furniture_df.iloc[index]['Image_Path']
image_arr = prepare(first_data)

prediction = model.predict(image_arr)
predicted_class_index = np.argmax(prediction, axis = 1)

# retrieve class label
predicted_class_label = cat_folders[predicted_class_index[0]]

# display classifying result
print('Prediction: ' + predicted_class_label)
print('Actual: ' + shuffled_furniture_df.iloc[index]['Furniture_Category'])


In [ ]:
#  multiple prediction
num_rows = (IMAGE_SHOW + 4) // 5
num_cols = min(IMAGE_SHOW, 5)
# empty list to store predictions and count correct predictions
predicted_result = []
correct = 0

fig, axes = plt.subplots(nrows = num_rows, ncols = num_cols, figsize = (16, 4 * num_rows))

def final_prediction(item1, item2):
    # test prediction
    return "Correct" if item1 == item2 else "Wrong"

# iterate through the image table
for i, ax in enumerate(axes.flat):
    # make predictions for the i-th image
    prediction = model.predict(image_test_list[i])
    predicted_class_label = cat_folders[np.argmax(prediction, axis = 1)[0]]
    predicted_result.append(predicted_class_label)

     # display the image
    ax.imshow(image_test_list[i][0])
    color = 'green' if predicted_result[i] == shuffled_furniture_df.loc[i, 'Furniture_Category'] else 'red'
    ax.set_title('Prediction: { }\n Actual: {}\n -- { } --'.format(predicted_result[i], 
                  shuffled_furniture_df.loc[i, 'Furniture_Category'], 
                  final_prediction(predicted_result[i], shuffled_furniture_df.loc[i, 'Furniture_Category'])), color = color)
    correct += (predicted_result[i] == shuffled_furniture_df.loc[i, 'Furniture_Category'])

plt.tight_layout()
plt.show()

print(f"Total correct predictions: {correct}/{IMAGE_SHOW}")
print(f"Percentage of correct predictions: {(correct * 100) / IMAGE_SHOW}%")